In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [4]:
dataset = pd.read_csv('BankCustomers.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### So 'Exited' column - dependent feature and all remains are - independent features. not reqd. like: Rownumber , CustomerId , Surname

In [5]:
X = dataset.iloc[:, 3:13]    # independent 
y = dataset.iloc[:, 13]      # dependent   

In [6]:
# Now we have Categorical features like: Geography , Gender , converting categorical feature into dummy var.

states = pd.get_dummies(X['Geography'], drop_first=True)    
gender = pd.get_dummies(X['Gender'], drop_first=True)

In [7]:
# dropping column 'Geography' , 'Gender', bcoz converted into dummy columns

X = X.drop(['Geography','Gender'],axis=1)  

In [8]:
X = pd.concat([X,states,gender],axis=1)     # concatenate columns of dummy var. to X

In [9]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [10]:
# Now split the dataset into training and testing dataset

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
# Scale down all features, Standard Normal Distribution (Mean=0 , Standard Deviation=1) scaled down b/w 0 to 1

from sklearn.preprocessing import StandardScaler

sc =StandardScaler()                             # scaling in X_train , X_test bcoz of different fetaures
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  import sys


### Building  ANN  model

#### we have 11 features as input layer, In 1st hidden layer 6 neurons and 2nd hidden layer also 6 neurons , output layer 1 feature

In [14]:
classifier = Sequential()

# input_dim - 11 input features, kernel_initializer - weights of neuron distributed uniformly
classifier.add(Dense(activation='relu', input_dim=11, units=6, kernel_initializer='uniform')) 

classifier.add(Dense(activation='relu', units=6, kernel_initializer='uniform'))  # 2nd hidden layer

classifier.add(Dense(activation='sigmoid', units=1, kernel_initializer='uniform'))  # sigmoid func. either value 0 or 1

classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


### Compile and fit the ANN

In [16]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  
# adam -reduce cost func. with it classify properly

# fit
classifier.fit(X_train,y_train, batch_size=10, epochs=50)  # epoch- combination of front and backward propagation

Epoch 1/50
8000/8000 [==============================] - 2s 293us/step - loss: 0.4033 - acc: 0.8347
Epoch 2/50
8000/8000 [==============================] - 1s 175us/step - loss: 0.4034 - acc: 0.8332
Epoch 3/50
8000/8000 [==============================] - 1s 163us/step - loss: 0.4027 - acc: 0.8346
Epoch 4/50
8000/8000 [==============================] - 1s 166us/step - loss: 0.4034 - acc: 0.8350
Epoch 5/50
8000/8000 [==============================] - 1s 174us/step - loss: 0.4030 - acc: 0.8334
Epoch 6/50
8000/8000 [==============================] - 2s 188us/step - loss: 0.4029 - acc: 0.8347
Epoch 7/50
8000/8000 [==============================] - 2s 189us/step - loss: 0.4020 - acc: 0.8346
Epoch 8/50
8000/8000 [==============================] - 2s 189us/step - loss: 0.4028 - acc: 0.8349
Epoch 9/50
8000/8000 [==============================] - 2s 195us/step - loss: 0.4028 - acc: 0.8349
Epoch 10/50
8000/8000 [==============================] - 2s 196us/step - loss: 0.4023 - acc: 0.8334
Epoch 11/

#### make Predictions

In [17]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)  # basically it means gives boolean value(True or False) of o and 1

#### Check performance

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

In [20]:
print('confusion matrix : \n', cm)       
print(f'\nAccuracy score : {accuracy : 0.2%}')

confusion matrix : 
 [[1539   56]
 [ 260  145]]

Accuracy score :  84.20%
